# Verifying depthwise convolutional

In [12]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import DepthwiseConv2D

from keras.optimizers import SGD
from keras.utils.generic_utils import CustomObjectScope

In [7]:
a = np.random.uniform(low=1, high=5, size=(3, 3, 5, 1))

In [8]:
a_ = a[0, 0, :, 0]

In [9]:
a_[3]

3.4857058692508596

In [10]:
from sklearn.externals import joblib

In [13]:
keras.__version__, tf.__version__

('2.2.4', '1.13.1')

In [14]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

In [13]:
myinp = Input(shape=(6, 6, 5), dtype='float32')

x = DepthwiseConv2D(kernel_size=3,
                        strides=1, activation=None,
                        use_bias=False, padding='same',
                        name='depthwise')(myinp)
model = Model(inputs=myinp, outputs=x, name='depth')

In [4]:
#model.get_weights()[0].shape, model.get_weights()[0]

In [15]:
sample_inp = np.random.uniform(low=5, high=10, size=(6, 6, 5))

In [5]:
#sample_inp.shape, sample_inp

In [17]:
with sess.as_default():
    outp = sess.run(model.output, feed_dict={model.input: sample_inp.reshape(1, 6, 6, 3), K.learning_phase(): 0})

ValueError: cannot reshape array of size 180 into shape (1,6,6,3)

In [18]:
outp.shape, outp

NameError: name 'outp' is not defined

In [19]:
BAS_PATH = '/home/coep/pkm/nn4small/mobNet_8137/'
np.save(BAS_PATH + 'depth_out.npy', outp)
np.save(BAS_PATH + 'depth_input.npy', sample_inp.reshape(1, 6, 6 ,3))
np.save(BAS_PATH + 'filter_weights.npy', model.get_weights()[0])

NameError: name 'outp' is not defined

# Generating weights for rpi

In [20]:
from keras.layers import DepthwiseConv2D, ReLU
relu6 = ReLU(6.)
# (np.random.randn(216,128), np.random.randn(216, 128))
def distill_mse(y_true, y_pred):
    return 0.5*K.mean(K.sum(K.square(y_true - K.reshape(y_pred, (-1, 128))), axis=1))
with CustomObjectScope({'relu6': keras.layers.ReLU(6.),'DepthwiseConv2D': keras.layers.DepthwiseConv2D}):
    mm = load_model('/home/coep/Desktop/mob_net_ep_10.h5', custom_objects={ 'distill_mse': distill_mse})

In [21]:
mm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 16)   64          Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (Activation)         (None, 48, 48, 16)   0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
mobl0_conv

In [22]:
mm.save_weights('/home/coep/Desktop/mob_net_10_weights.h5')

## Verifying Depthwise from mob_net_ep_5

In [15]:
from keras.layers import DepthwiseConv2D, ReLU
relu6 = ReLU(6.)
# (np.random.randn(216,128), np.random.randn(216, 128))
def distill_mse(y_true, y_pred):
    return 0.5*K.mean(K.sum(K.square(y_true - K.reshape(y_pred, (-1, 128))), axis=1))
with CustomObjectScope({'relu6': keras.layers.ReLU(6.),'DepthwiseConv2D': keras.layers.DepthwiseConv2D}):
    mm = load_model('/home/coep/pkm/dist_models_backup/adam/iter_5/mob_net_ep_19.h5', custom_objects={ 'distill_mse': distill_mse})

In [16]:
mm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 16)   64          Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (Activation)         (None, 48, 48, 16)   0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
mobl0_conv

In [31]:
import cv2
img = cv2.resize(cv2.imread('/home/coep/Downloads/sample.jpeg'), (96, 96)).reshape(1, 96, 96, 3)
img = np.around(img/255.0, decimals=6)


In [32]:
with tf.device('/gpu:0'):
    with sess.as_default():
        fd = {mm.input: img, K.learning_phase(): 0}
        outp = sess.run(mm.get_layer("Conv1").output, feed_dict=fd)
        

array([[ -6.4808927,  -6.9985785,  -9.142585 , ..., -10.649666 ,
         -9.091619 , -18.572725 ],
       [ -7.22959  ,  -8.191075 , -10.55305  , ..., -10.451361 ,
         -8.934333 , -18.360107 ],
       [ -8.542431 ,  -9.064327 , -10.79018  , ..., -11.623117 ,
        -11.043085 , -19.624535 ],
       ...,
       [-14.530853 , -13.54793  , -14.937292 , ..., -10.849326 ,
        -11.133446 , -20.13433  ],
       [-14.158226 , -14.601536 , -13.905693 , ..., -11.892778 ,
        -12.17492  , -20.49289  ],
       [-18.981888 , -20.736776 , -20.063896 , ..., -17.158306 ,
        -17.586378 , -20.357996 ]], dtype=float32)

In [47]:
mm.get_layer("Conv1").get_weights()[0][1,:,0,0]

array([1.1718539e-21, 6.2277446e-22, 6.2802239e-22], dtype=float32)

In [22]:
mm.get_layer("Conv1").get_weights()[0][:,:,1,10]

array([[-3.4272864,  3.803323 ,  1.8229194],
       [-4.6645017,  5.6499324,  2.7575119],
       [-1.2039093,  3.5220745,  4.988641 ]], dtype=float32)

In [26]:
sum(sum(sum(mm.get_layer("Conv1").get_weights()[0][:,:,:,10])))

40.91922664642334

In [24]:
mm.get_layer("mobl2_conv_expand").get_weights()[0].flatten().shape[0]

1536

In [8]:
c = []
for layer in mm.layers:
    if len(mm.get_layer(layer.name).get_weights()):
        c.append(1)
        print("Name: {0}; len: {1} size:{2}".format(layer.name, len(mm.get_layer(layer.name).get_weights()), mm.get_layer(layer.name).get_weights()[0].flatten().shape[0]))
print(len(c))

Name: Conv1; len: 1 size:432
Name: bn_Conv1; len: 4 size:16
Name: mobl0_conv_depthwise; len: 1 size:144
Name: bn0_conv_depthwise; len: 4 size:16
Name: mobl0_conv_project; len: 1 size:128
Name: bn0_conv_project; len: 4 size:8
Name: mobl1_conv_expand; len: 1 size:384
Name: bn1_conv_bn_expand; len: 4 size:48
Name: mobl1_conv_depthwise; len: 1 size:432
Name: bn1_conv_depthwise; len: 4 size:48
Name: mobl1_conv_project; len: 1 size:768
Name: bn1_conv_bn_project; len: 4 size:16
Name: mobl2_conv_expand; len: 1 size:1536
Name: bn2_conv_bn_expand; len: 4 size:96
Name: mobl2_conv_depthwise; len: 1 size:864
Name: bn2_conv_depthwise; len: 4 size:96
Name: mobl2_conv_project; len: 1 size:1536
Name: bn2_conv_bn_project; len: 4 size:16
Name: mobl3_conv_expand; len: 1 size:1536
Name: bn3_conv_bn_expand; len: 4 size:96
Name: mobl3_conv_depthwise; len: 1 size:864
Name: bn3_conv_depthwise; len: 4 size:96
Name: mobl3_conv_project; len: 1 size:1536
Name: bn3_conv_bn_project; len: 4 size:16
Name: mobl4_conv_e

In [ ]:
with sess.as_default():
    sess.run(mm.get_layer("mobl1_conv_depthwise").output, )